# Estimacion DSGE Smets & Wouters - Argentina

Este notebook replica las **Tablas 1A y 1B** del modelo Smets & Wouters (2007) aplicado a datos de Argentina.

- **Tabla 1A**: Distribucion Prior y Posterior de Parametros Estructurales (19 parametros)
- **Tabla 1B**: Distribucion Prior y Posterior de Procesos de Shock (17 parametros)

**Periodo de estimacion**: 2004Q2 - 2025Q3 (86 observaciones)

**Requisitos**:
- GNU Octave instalado
- Dynare 6.x instalado
- Datos preparados en `argmodel_data.mat`

## 1. Setup y Configuracion

In [1]:
# Imports
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

# Agregar directorio padre al path
sys.path.append(str(Path.cwd().parent))

from src import DynareInterface

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Imports completados")

Imports completados


In [2]:
# Configurar rutas - MODIFICAR SEGUN TU INSTALACION
os.environ['OCTAVE_EXECUTABLE'] = r'C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe'

DYNARE_PATH = r'C:\dynare\6.5\matlab'
MODEL_PATH = Path.cwd().parent / 'model'
DATA_PATH = Path.cwd().parent / 'data'
OUTPUT_PATH = Path.cwd().parent / 'output'

print(f"Octave executable: {os.environ['OCTAVE_EXECUTABLE']}")
print(f"Dynare path: {DYNARE_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Data path: {DATA_PATH}")
print(f"Model exists: {MODEL_PATH.exists()}")

Octave executable: C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe
Dynare path: C:\dynare\6.5\matlab
Model path: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\DG\model
Data path: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\DG\data
Model exists: True


## 2. Preparar Datos (si es necesario)

Ejecutar solo si `argmodel_data.mat` no existe.

In [3]:
# Verificar si los datos ya estan preparados
mat_file = DATA_PATH / 'argmodel_data.mat'

if not mat_file.exists():
    print("Preparando datos de Argentina...")
    from src.data_preparation import prepare_argentina_data
    prepare_argentina_data()
    print("Datos preparados!")
else:
    print(f"Datos ya existen: {mat_file}")

Datos ya existen: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\DG\data\argmodel_data.mat


## 3. Definiciones de Parametros

Definiciones de los parametros estructurales y de shocks siguiendo la nomenclatura de Smets & Wouters (2007).

In [4]:
# =============================================================================
# TABLA 1A - PARAMETROS ESTRUCTURALES
# =============================================================================

TABLE_1A_PARAMS = {
    # Parameter: (Dynare name, Prior Distr, Prior Mean, Prior SD, Description)
    'phi':       ('csadjcost', 'Normal',  4.00, 1.50, 'Costo ajuste inversion'),
    'sigma_c':   ('csigma',    'Normal',  1.50, 0.37, 'Aversion al riesgo'),
    'h':         ('chabb',     'Beta',    0.70, 0.10, 'Formacion de habito'),
    'xi_w':      ('cprobw',    'Beta',    0.50, 0.10, 'Prob. Calvo salarios'),
    'sigma_l':   ('csigl',     'Normal',  2.00, 0.75, 'Elasticidad oferta laboral'),
    'xi_p':      ('cprobp',    'Beta',    0.50, 0.10, 'Prob. Calvo precios'),
    'iota_w':    ('cindw',     'Beta',    0.50, 0.15, 'Indexacion salarios'),
    'iota_p':    ('cindp',     'Beta',    0.50, 0.15, 'Indexacion precios'),
    'psi':       ('czcap',     'Beta',    0.50, 0.15, 'Utilizacion capacidad'),
    'Phi':       ('cfc',       'Normal',  1.25, 0.12, 'Costo fijo'),
    'r_pi':      ('crpi',      'Normal',  1.50, 0.25, 'Taylor: inflacion'),
    'rho':       ('crr',       'Beta',    0.75, 0.10, 'Taylor: suavizamiento'),
    'r_y':       ('cry',       'Normal',  0.12, 0.05, 'Taylor: output gap'),
    'r_Delta_y': ('crdy',      'Normal',  0.12, 0.05, 'Taylor: crecimiento output'),
    'pi_bar':    ('constepinf','Gamma',   0.62, 0.10, 'Inflacion estado estacionario'),
    'beta_const':('constebeta','Gamma',   0.25, 0.10, '100(beta^-1 - 1)'),
    'l_bar':     ('constelab', 'Normal',  0.00, 2.00, 'Horas estado estacionario'),
    'gamma_bar': ('ctrend',    'Normal',  0.40, 0.10, 'Tendencia crecimiento'),
    'alpha':     ('calfa',     'Normal',  0.30, 0.05, 'Participacion capital'),
}

print(f"Tabla 1A: {len(TABLE_1A_PARAMS)} parametros estructurales definidos")

Tabla 1A: 19 parametros estructurales definidos


In [5]:
# =============================================================================
# TABLA 1B - PROCESOS DE SHOCK
# =============================================================================

TABLE_1B_PARAMS = {
    # Desviaciones estandar de shocks
    'sigma_a':   ('ea',        'InvGamma', 0.10, 2.00, 'Shock tecnologia'),
    'sigma_b':   ('eb',        'InvGamma', 0.10, 2.00, 'Shock preferencia'),
    'sigma_g':   ('eg',        'InvGamma', 0.10, 2.00, 'Shock gasto gobierno'),
    'sigma_I':   ('eqs',       'InvGamma', 0.10, 2.00, 'Shock inversion'),
    'sigma_r':   ('em',        'InvGamma', 0.10, 2.00, 'Shock politica monetaria'),
    'sigma_p':   ('epinf',     'InvGamma', 0.10, 2.00, 'Shock markup precios'),
    'sigma_w':   ('ew',        'InvGamma', 0.10, 2.00, 'Shock markup salarios'),
    
    # Coeficientes AR(1)
    'rho_a':     ('crhoa',     'Beta',     0.50, 0.20, 'Persistencia tecnologia'),
    'rho_b':     ('crhob',     'Beta',     0.50, 0.20, 'Persistencia preferencia'),
    'rho_g':     ('crhog',     'Beta',     0.50, 0.20, 'Persistencia gasto'),
    'rho_I':     ('crhoqs',    'Beta',     0.50, 0.20, 'Persistencia inversion'),
    'rho_r':     ('crhoms',    'Beta',     0.50, 0.20, 'Persistencia pol. monetaria'),
    'rho_p':     ('crhopinf',  'Beta',     0.50, 0.20, 'Persistencia markup precios'),
    'rho_w':     ('crhow',     'Beta',     0.50, 0.20, 'Persistencia markup salarios'),
    
    # Coeficientes MA y cross-effect
    'mu_p':      ('cmap',      'Beta',     0.50, 0.20, 'MA markup precios'),
    'mu_w':      ('cmaw',      'Beta',     0.50, 0.20, 'MA markup salarios'),
    'rho_ga':    ('cgy',       'Beta',     0.50, 0.20, 'Efecto cruzado g-a'),
}

print(f"Tabla 1B: {len(TABLE_1B_PARAMS)} parametros de shocks definidos")

Tabla 1B: 17 parametros de shocks definidos


## 4. Ejecutar Dynare y Extraer Parametros

In [6]:
# Inicializar interfaz Dynare
di = DynareInterface(DYNARE_PATH, str(MODEL_PATH))
print("Interfaz Dynare inicializada")

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

Interfaz Dynare inicializada


In [7]:
# Ejecutar modelo
print("Ejecutando argmodel.mod...")
print("(Esto puede tardar varios minutos debido a la estimacion MCMC)\n")

di.run_model('argmodel.mod')

print("\nDynare completado")

Ejecutando argmodel.mod...
(Esto puede tardar varios minutos debido a la estimacion MCMC)


Step 1: Closing Octave session to release file locks...
Waiting for Windows to release file handles...

Step 2: Cleaning up directories...
Searching for directories to clean up...
Adding model directory to cleanup: argmodel/
Found 1 directory(ies): ['argmodel']
  Retry 1/5 for argmodel...
  Retry 2/5 for argmodel...
  Retry 3/5 for argmodel...
  Retry 4/5 for argmodel...
  Failed to remove argmodel with shutil, trying Windows command...
Cleaned up directory with rmdir: argmodel

Step 3: Starting fresh Octave session...
    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

    _pyeval at line 57 column 10

Octave session ready

Step 4: Running Dynare estimation...
Command: dynare argmodel nograph
(This may take several minutes...)

Starting Dynare (version 6.5).
Calling Dynare with arguments: nograph
Starting preprocessing of the model file ...
Found 40 equation(s).
Evaluating expre

In [8]:
# Extraer parametros estimados (posterior mode)
params_df = di.get_parameters()

# Convertir a diccionario
estimated_params = dict(zip(params_df['parameter'], params_df['value']))

print(f"Extraidos {len(estimated_params)} parametros de Dynare")
print("\nParametros clave:")
for param in ['csigma', 'chabb', 'cprobw', 'cprobp', 'crpi', 'crr', 'calfa']:
    if param in estimated_params:
        print(f"  {param}: {estimated_params[param]:.4f}")

Extraidos 57 parametros de Dynare

Parametros clave:
  csigma: 1.1391
  chabb: 0.8891
  cprobw: 0.8988
  cprobp: 0.8622
  crpi: 1.7018
  crr: 0.8973
  calfa: 0.6181


In [9]:
# Extraer desviaciones estandar de shocks
def extract_shock_std_devs(di):
    """Extrae desviaciones estandar de shocks de Dynare."""
    shock_stds = {}
    
    n_shocks = int(di.oc.eval('M_.exo_nbr', nout=1))
    shock_names = []
    for i in range(n_shocks):
        name = di.oc.eval(f'deblank(M_.exo_names{{{i+1}}})', nout=1)
        shock_names.append(str(name).strip())
    
    try:
        has_shocks_std = di.oc.eval(
            'isfield(oo_, "posterior_mode") && isfield(oo_.posterior_mode, "shocks_std")',
            nout=1
        )
        if has_shocks_std:
            shocks_std = di.oc.eval('struct2array(oo_.posterior_mode.shocks_std)', nout=1)
            if hasattr(shocks_std, 'flatten'):
                shocks_std = shocks_std.flatten()
            for i, name in enumerate(shock_names):
                if i < len(shocks_std):
                    shock_stds[name] = float(shocks_std[i])
            print("Desviaciones de shocks extraidas de oo_.posterior_mode.shocks_std")
            return shock_stds
    except Exception as e:
        print(f"Error: {e}")
    
    # Fallback: usar M_.Sigma_e
    try:
        sigma_e = di.oc.eval('M_.Sigma_e', nout=1)
        for i, name in enumerate(shock_names):
            shock_stds[name] = float(np.sqrt(sigma_e[i, i]))
        print("Desviaciones de shocks extraidas de M_.Sigma_e")
    except:
        pass
    
    return shock_stds

shock_std_estimates = extract_shock_std_devs(di)

if shock_std_estimates:
    print("\nDesviaciones estandar de shocks estimadas:")
    for name, val in shock_std_estimates.items():
        print(f"  {name}: {val:.4f}")

Desviaciones de shocks extraidas de oo_.posterior_mode.shocks_std

Desviaciones estandar de shocks estimadas:
  ea: 1.2368
  eb: 0.3198
  eg: 0.7323
  eqs: 0.7513
  em: 0.3720
  epinf: 0.4186
  ew: 0.3143


In [10]:
# Verificar resultados MCMC (media posterior e intervalos)
def check_mcmc_results(di):
    """Verifica si hay resultados MCMC disponibles."""
    mcmc_results = {
        'available': False,
        'param_means': {},
        'param_intervals': {},
        'shock_means': {},
        'shock_intervals': {}
    }
    
    try:
        has_mcmc = di.oc.eval(
            'isfield(oo_, "posterior_mean") && isfield(oo_.posterior_mean, "parameters")',
            nout=1
        )
        
        if has_mcmc:
            mcmc_results['available'] = True
            print("Resultados MCMC disponibles")
            
            # Extraer medias
            n_params = int(di.oc.eval('length(fieldnames(oo_.posterior_mean.parameters))', nout=1))
            param_names_est = []
            for i in range(n_params):
                name = di.oc.eval(f'deblank(M_.param_names{{estim_params_.param_vals({i+1},1)}})', nout=1)
                param_names_est.append(str(name).strip())
            
            means = di.oc.eval('struct2array(oo_.posterior_mean.parameters)', nout=1).flatten()
            for i, name in enumerate(param_names_est):
                if i < len(means):
                    mcmc_results['param_means'][name] = float(means[i])
            
            # Extraer intervalos HPD
            try:
                has_hpd = di.oc.eval('isfield(oo_, "posterior_hpdinf") && isfield(oo_.posterior_hpdinf, "parameters")', nout=1)
                if has_hpd:
                    hpd_inf = di.oc.eval('struct2array(oo_.posterior_hpdinf.parameters)', nout=1).flatten()
                    hpd_sup = di.oc.eval('struct2array(oo_.posterior_hpdsup.parameters)', nout=1).flatten()
                    for i, name in enumerate(param_names_est):
                        if i < len(hpd_inf):
                            mcmc_results['param_intervals'][name] = (float(hpd_inf[i]), float(hpd_sup[i]))
            except:
                pass
        else:
            print("Resultados MCMC NO disponibles (mh_replic=0)")
            
    except Exception as e:
        print(f"Error verificando MCMC: {e}")
    
    return mcmc_results

mcmc_results = check_mcmc_results(di)

Resultados MCMC NO disponibles (mh_replic=0)


## 5. Generar Tabla 1A - Parametros Estructurales

In [11]:
def create_table_1A(param_defs, estimated_params, mcmc_results=None):
    """Crea Tabla 1A con parametros estructurales."""
    rows = []
    
    for param_symbol, values in param_defs.items():
        dynare_name, prior_distr, prior_mean, prior_sd, description = values
        
        estimated_mode = estimated_params.get(dynare_name, np.nan)
        
        estimated_mean = np.nan
        estimated_5 = np.nan
        estimated_95 = np.nan
        
        if mcmc_results and mcmc_results['available']:
            estimated_mean = mcmc_results['param_means'].get(dynare_name, np.nan)
            if dynare_name in mcmc_results['param_intervals']:
                estimated_5, estimated_95 = mcmc_results['param_intervals'][dynare_name]
        
        rows.append({
            'Parametro': param_symbol,
            'Nombre Dynare': dynare_name,
            'Descripcion': description,
            'Prior Distr.': prior_distr,
            'Prior Mean': prior_mean,
            'Prior SD': prior_sd,
            'Post. Mode': estimated_mode,
            'Post. Mean': estimated_mean,
            'Post. 5%': estimated_5,
            'Post. 95%': estimated_95,
        })
    
    return pd.DataFrame(rows)

table_1A = create_table_1A(TABLE_1A_PARAMS, estimated_params, mcmc_results)
print("Tabla 1A generada")

Tabla 1A generada


In [12]:
# Mostrar Tabla 1A
print("="*90)
print("TABLA 1A - DISTRIBUCION PRIOR Y POSTERIOR DE PARAMETROS ESTRUCTURALES")
print("Modelo DSGE Smets & Wouters - Argentina (2004Q2-2025Q3)")
print("="*90)

display_cols = ['Parametro', 'Prior Distr.', 'Prior Mean', 'Prior SD', 'Post. Mode', 'Post. Mean', 'Post. 5%', 'Post. 95%']
print(table_1A[display_cols].to_string(index=False, float_format=lambda x: f'{x:.4f}' if pd.notna(x) else 'N/A'))

TABLA 1A - DISTRIBUCION PRIOR Y POSTERIOR DE PARAMETROS ESTRUCTURALES
Modelo DSGE Smets & Wouters - Argentina (2004Q2-2025Q3)
 Parametro Prior Distr.  Prior Mean  Prior SD  Post. Mode  Post. Mean  Post. 5%  Post. 95%
       phi       Normal      4.0000    1.5000      6.3450         NaN       NaN        NaN
   sigma_c       Normal      1.5000    0.3700      1.1391         NaN       NaN        NaN
         h         Beta      0.7000    0.1000      0.8891         NaN       NaN        NaN
      xi_w         Beta      0.5000    0.1000      0.8988         NaN       NaN        NaN
   sigma_l       Normal      2.0000    0.7500      2.8461         NaN       NaN        NaN
      xi_p         Beta      0.5000    0.1000      0.8622         NaN       NaN        NaN
    iota_w         Beta      0.5000    0.1500      0.4088         NaN       NaN        NaN
    iota_p         Beta      0.5000    0.1500      0.2525         NaN       NaN        NaN
       psi         Beta      0.5000    0.1500      0.61

## 6. Generar Tabla 1B - Procesos de Shock

In [13]:
def create_table_1B(param_defs, estimated_params, shock_std_estimates, mcmc_results=None):
    """Crea Tabla 1B con parametros de shocks."""
    rows = []
    
    for param_symbol, values in param_defs.items():
        dynare_name, prior_distr, prior_mean, prior_sd, description = values
        
        is_shock_std = param_symbol.startswith('sigma_')
        
        if is_shock_std:
            estimated_mode = shock_std_estimates.get(dynare_name, np.nan)
        else:
            estimated_mode = estimated_params.get(dynare_name, np.nan)
        
        estimated_mean = np.nan
        if mcmc_results and mcmc_results['available']:
            if is_shock_std:
                estimated_mean = mcmc_results.get('shock_means', {}).get(dynare_name, np.nan)
            else:
                estimated_mean = mcmc_results['param_means'].get(dynare_name, np.nan)
        
        rows.append({
            'Parametro': param_symbol,
            'Nombre Dynare': dynare_name,
            'Descripcion': description,
            'Tipo': 'Shock Std' if is_shock_std else 'Parametro',
            'Prior Distr.': prior_distr,
            'Prior Mean': prior_mean,
            'Prior SD': prior_sd,
            'Post. Mode': estimated_mode,
            'Post. Mean': estimated_mean,
        })
    
    return pd.DataFrame(rows)

table_1B = create_table_1B(TABLE_1B_PARAMS, estimated_params, shock_std_estimates, mcmc_results)
print("Tabla 1B generada")

Tabla 1B generada


In [14]:
# Mostrar Tabla 1B
print("="*90)
print("TABLA 1B - DISTRIBUCION PRIOR Y POSTERIOR DE PROCESOS DE SHOCK")
print("Modelo DSGE Smets & Wouters - Argentina (2004Q2-2025Q3)")
print("="*90)

display_cols = ['Parametro', 'Tipo', 'Prior Distr.', 'Prior Mean', 'Prior SD', 'Post. Mode', 'Post. Mean']
print(table_1B[display_cols].to_string(index=False, float_format=lambda x: f'{x:.4f}' if pd.notna(x) else 'N/A'))

TABLA 1B - DISTRIBUCION PRIOR Y POSTERIOR DE PROCESOS DE SHOCK
Modelo DSGE Smets & Wouters - Argentina (2004Q2-2025Q3)
Parametro      Tipo Prior Distr.  Prior Mean  Prior SD  Post. Mode  Post. Mean
  sigma_a Shock Std     InvGamma      0.1000    2.0000      1.2368         NaN
  sigma_b Shock Std     InvGamma      0.1000    2.0000      0.3198         NaN
  sigma_g Shock Std     InvGamma      0.1000    2.0000      0.7323         NaN
  sigma_I Shock Std     InvGamma      0.1000    2.0000      0.7513         NaN
  sigma_r Shock Std     InvGamma      0.1000    2.0000      0.3720         NaN
  sigma_p Shock Std     InvGamma      0.1000    2.0000      0.4186         NaN
  sigma_w Shock Std     InvGamma      0.1000    2.0000      0.3143         NaN
    rho_a Parametro         Beta      0.5000    0.2000      0.7197         NaN
    rho_b Parametro         Beta      0.5000    0.2000      0.4102         NaN
    rho_g Parametro         Beta      0.5000    0.2000      0.9181         NaN
    rho_I Pa

## 7. Exportar Resultados

In [15]:
# Exportar tablas a CSV
tables_dir = OUTPUT_PATH / 'tables'
tables_dir.mkdir(parents=True, exist_ok=True)

table_1A.to_csv(tables_dir / 'table_1A_argentina.csv', index=False)
print(f"Tabla 1A guardada en: {tables_dir / 'table_1A_argentina.csv'}")

table_1B.to_csv(tables_dir / 'table_1B_argentina.csv', index=False)
print(f"Tabla 1B guardada en: {tables_dir / 'table_1B_argentina.csv'}")

Tabla 1A guardada en: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\DG\output\tables\table_1A_argentina.csv
Tabla 1B guardada en: c:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECTMT11\DG\output\tables\table_1B_argentina.csv


## 8. Cleanup

In [16]:
# Cerrar sesion Octave
di.close()
print("Sesion Octave cerrada")
print("\nEstimacion completada!")

Sesion Octave cerrada

Estimacion completada!
